# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from IPython.display import display, HTML #used for output tables formatting

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="table.png">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster()
    session = cluster.connect()
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
#session = cluster.connect()

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace
session.execute( """
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION = 
{'class': 'SimpleStrategy', 'replication_factor': 1}"""
)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#Creating an empy table if it does not already exist, and then specifying columnn names and accepatble types of data that will be 
#injected into these columns
#Only including columns to that table that directly relate to the query 
#Specifying the primary key to make sure it is unique, make sure that the columns that are included in the where-clause of the query
#are part of the primary key
query = 'CREATE TABLE IF NOT EXISTS music_app_history'
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
# since sessionId and itemInSession are in the where clause, they are part of the primary key, and since the combination of the 
# two gives a unique primary key (by observation), no other additions to primary keys are needed.

#executing the above query
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

#Populating the values from CSV file into the columns that exist in the above table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history(sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query="select artist, song, length from music_app_history where sessionId=338 and itemInSession=4"
try:
    rows=session.execute(query)
    data=[dict(row._asdict()) for row in rows]  #sort data into a dictionary
    df=pd.DataFrame(data) #output data as a table
    df.columns = df.columns.str.title() #make column titles start from capital letter
    print("\nAll of the artists, song titles and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 are")
   # Display with bold titles, and remove the first column that contains of indexes
    display(HTML(df.to_html(header=True, escape=False, index=False))) 
except Exception as e:
    print(e)

#for row in rows:
    #print(row.artist, row.song, row.length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### Query 2

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#Creating an empy table if it does not already exist, and then specifying columnn names and accepatble types of data that will be 
#injected into these columns
#Only including columns to that table that directly relate to the query 
#Specifying the primary key to make sure it is unique, make sure that the columns that are included in the where-clause of the query
#are part of the primary key
query = 'CREATE TABLE IF NOT EXISTS user_info'
query = query + "(userId int, sessionId int, artist text, song text, itemInSession int, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"
#(userId, sessionId) - partition key -  ensures that the data for each unique combination of userId and sessionId is grouped together,
#itemInSession - clustering key - determines the order in which the data is stored within a partition
# - within each partition (for a specific userId and sessionId), the data will be sorted by itemInSession

#executing the above query
try:
    session.execute(query)
except Exception as e:
    print(e)


In [ ]:
#Populating the values from CSV file into the columns that exist in the above table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_info(userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query="select artist, song, firstName, lastName from user_info where userId=10 and sessionId=182"
try:
    rows=session.execute(query)
    data=[dict(row._asdict()) for row in rows]
    df=pd.DataFrame(data)
    df.columns = df.columns.str.title()
    print("\n Artist name, song and name of the user for userid = 10, sessionid = 182 are")
    # Display with bold titles
    display(HTML(df.to_html(header=True, escape=False, index=False))) # Reset the index and drop the old index column
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

### Query 3

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#Creating an empy table if it does not already exist, and then specifying columnn names and accepatble types of data that will be 
#injected into these columns
#Only including columns to that table that directly relate to the query 
#Specifying the primary key to make sure it is unique, make sure that the columns that are included in the where-clause of the query
#are part of the primary key
query = 'CREATE TABLE IF NOT EXISTS song_lovers'
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"
#song column is in the where-clause, so it is a part of primary key and together with userId
#it gives a unique primary key

#executing the above query
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [ ]:
#Populating the values from CSV file into the columns that exist in the above table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_lovers(song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query="select firstName, lastName from song_lovers where song='All Hands Against His Own'"
#song - primary key so can use where-clause on it
try:
    rows=session.execute(query)
    names=[dict(row._asdict()) for row in rows]
    df=pd.DataFrame(names)
    df = df.rename(columns={
        'firstname': 'First Name',  # Rename 'firstName' to 'First Name'
        'lastname': 'Last Name'    # Rename 'lastName' to 'Last Name'
    })
    print("\nList of Song Lovers for 'All Hands Against His Own':")
     # Display with bold titles
    display(HTML(df.to_html(header=True, escape=False, index=False))) # Reset the index and drop the old index column
except Exception as e:
    print(e) 

#for row in rows:
    #print(row.firstname, row.lastname)

### Drop the tables before closing out the sessions

In [ ]:
query1 =  "DROP TABLE music_app_history"
try:
    rows = session.execute(query1)
    print('Table 1 is dropped.')
except Exception as e:
    print(e)
    
query2 =  "DROP TABLE user_info"
try:
    rows = session.execute(query2)
    print('Table 2 is dropped.')
except Exception as e:
    print(e)
    
query3 =  "DROP TABLE song_lovers"
try:
    rows = session.execute(query3)
    print('Table 3 is dropped.')
except Exception as e:
    print(e)

### Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()